In [153]:
import pandas as pd
import numpy as np
import math

Read the file

In [122]:
file = pd.read_excel('shortcourses2566.xlsx')

Create Series of users

In [3]:
s_name = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
Users = pd.Series(s_name, name='Users')
Users

0             PORPHAING JANTIP
1         THANAKORN DARASRISAK
2             KARNJANA EAMTANG
3             KARNJANA EAMTANG
4       THITIMUN VORATHONGCHAI
                 ...          
6122         KITTINON SANTASUP
6123      WITTAWAT SERMSRIPONG
6124               NON NAKKARA
6125         PHATTHARAPHON LIN
6126      THANAPAT SEEMAKAJOHN
Name: Users, Length: 6127, dtype: object

Create Series of emails

In [5]:
s_email = file.loc[:, 'อีเมล'].fillna("")
Emails = pd.Series(s_email ,name='Emails')
Emails


0            tanghaoren17@gamil.com
1               tuakung21@gmail.com
2            karnjana.aon@gmail.com
3            karnjana.aon@gmail.com
4          thitimun.rama1@gmail.com
                   ...             
6122           kittinon_s@cmu.ac.th
6123        wittawat_serm@cmu.ac.th
6124          non_nakkara@cmu.ac.th
6125    phattharaphon_lin@cmu.ac.th
6126         thanapat_see@cmu.ac.th
Name: Emails, Length: 6127, dtype: object

Provide a score to each user based on their email domain

In [6]:
email_score = []
for data in Emails:
    if data != '':
        if data.split('@')[1] == 'cmu.ac.th':
            email_score.append(2)
        else:
            email_score.append(1)
    else:
        email_score.append(0)
email_score = pd.Series(email_score, name='Score Emails')
email_score

0       1
1       1
2       1
3       1
4       1
       ..
6122    2
6123    2
6124    2
6125    2
6126    2
Name: Score Emails, Length: 6127, dtype: int64

Create function to calculate age-education score

In [4]:
def getAgeEducationScore(age, limit_age):
    if age <= limit_age:
        score = 1
    else:
        score = 3
    return score

Create Series of Age-Education

In [162]:
ages = file.loc[:, 'อายุ']
educations = file.loc[:, 'วุฒิการศึกษา']
age_education_scores = []
# check output length of age_education_scores until it has 6127
for i,x in enumerate(educations):
    if  x == 'อื่นๆ (-)' or pd.isna(x) :
        age_education_scores.append(getAgeEducationScore(ages[i], 0))
    elif x == 'ประถมศึกษา' or x == 'อื่นๆ (ป.4)' or x == 'อื่นๆ (ป.7)' or x == 'อื่นๆ (ป7)':
        age_education_scores.append(getAgeEducationScore(ages[i], 16))
    elif x == 'มัธยมศึกษาตอนต้น' or x == 'Secondary school' or x == 'อื่นๆ (มศ.3)':
        age_education_scores.append(getAgeEducationScore(ages[i], 19))
    elif x == 'มัธยมศึกษาตอนปลาย' or x == 'High school' or x == 'Vocational' or x == 'การศึกษานอกระบบ' \
            or x == 'ประกาศนียบัตรวิชาชีพ (ปวช.)' or x == 'อื่นๆ (ม.ปลาย จบหลักสูตรEMR เป็นจนท.ปฏิบัติการ)'\
            or x == 'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาตอนปลาย)' or x == 'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาปีที่6)' \
            or x == 'อื่นๆ (มศ.5)' :
        age_education_scores.append(getAgeEducationScore(ages[i], 22))
    elif x == 'ปริญญาตรี' or x == 'Bachelor degree' or x == 'Diploma' or x == 'High Vocational' \
            or x == 'ประกาศนียบัตรวิชาชีพชั้นสูง (ปวส.)' or x == 'อื่นๆ (กำลังศึกษาในระดับปริญญาตรี)' \
            or x == 'อื่นๆ (กำลังศึกษาปริญญาตรี สาขารังสีเทคนิค)' or x == 'อื่นๆ (ปริญญาแพทยศาสตร์บัณฑิต)' \
            or x == 'อื่นๆ (นักศึกษาแพทย์ปี 5)' or x == 'อื่นๆ (นักศึกษาแพทย์ มช ปี4 ศูนย์เชียงราย)' \
            or x == 'อื่นๆ (แพทยศาสตร์บัณฑิต)' or x == 'อื่นๆ (แพทย์)'  or x == 'อื่นๆ (ประกาศณียบัตรผู้ช่วยพยาบาล)' \
            or x == 'อนุปริญญา' or x == 'อื่นๆ (ป.ตรี)' or x == 'อื่นๆ (ผู้ช่วยพยาบาล)':
        age_education_scores.append(getAgeEducationScore(ages[i], 26))
    elif x == 'ปริญญาโท' or x == 'ปริญญาเอก' or x == "Master's degree" or x == 'Other (OBGYN specalist lavel 1)' \
            or x == 'Other (Residency)' or x == 'Ph.D.' or x == 'อื่นๆ (Internal Medicine)' \
            or x == 'อื่นๆ (เฉพาะทาง)' or x == ('อื่นๆ (วุฒิบัตร)') or x == 'อื่นๆ (วว.ออร์โธปิดิกส์)' \
            or x == 'อื่นๆ (วุฒิบัตรแสดงความรู้ความชำนาญในการประกอบวิชาชีพเภสัชกรรม สาขาเภสัชบำบัด)'\
            or x == 'อื่นๆ (วุฒิบัตรผู้เชี่ยวชาญสาขาทันตกรรมทั่วไป)' or x == 'อื่นๆ (วุฒิบัตรศัลยศาสตร์และแม็กซิลโลเฟเชียล)':
        age_education_scores.append(getAgeEducationScore(ages[i], 40))
print(len(age_education_scores))


6127


Create Series of status

In [97]:
status = file.loc[:, 'สถานะ'].fillna("")
status = pd.Series(status ,name='status')

Provide a score to each user based on their purchase status

In [49]:
status_score = []
for x in status:
    if x == 'ชำระเงิน':
        status_score.append(8)
    if x == 'ไม่ผ่านการอนุมัติ':
        status_score.append(7)
    if x == 'ค้างชำระ':
        status_score.append(5)

Create Series of address

In [20]:
address = file.loc[:, 'ที่อยู่'].fillna("")
address = pd.Series(address ,name='status')

Provide a score to each user based on their providing address information or not

In [29]:
address_score = []
for x in address:
    if x == "":
        address_score.append(3)
    else:
        address_score.append(4)
address_score

[3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
